In [ ]:
# Load in software packages
%matplotlib inline
from parcels import FieldSet, Field, ParticleFile, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, plotTrajectoriesFile, Variable
from parcels import ErrorCode
import numpy as np
import math
from parcels import rng as random
from datetime import timedelta, datetime
from operator import attrgetter
import cartopy
import os
from glob import glob

In [ ]:
filenames = {'U':"chapter_4_models/copernicus_phy001_030/global-reanalysis-phy-001-030-daily_*.nc",
             'V':"chapter_4_models/copernicus_phy001_030/global-reanalysis-phy-001-030-daily_*.nc",
             'bathy':'chapter_4_models/copernicus_phy001_030_Statics/GLO-MFC_001_030_mask_bathy.nc',
             'mask':'chapter_4_models/copernicus_phy001_030_Statics/GLO-MFC_001_030_mask_bathy.nc',
             'S':"chapter_4_models/copernicus_phy001_030/global-reanalysis-phy-001-030-daily_*.nc"}
variables = {'U':'uo',
             'V':'vo',
             'S':'so',
             'mask':'mask',
            'bathy':'deptho'}
dimensions = {'U': {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'},
              'V': {'lon': 'longitude', 'lat': 'latitude','time': 'time'},
              'S': {'lon': 'longitude', 'lat': 'latitude','time': 'time'},
              'mask': {'lon': 'longitude', 'lat': 'latitude', 'depth':'depth'},
              'bathy': {'lon': 'longitude', 'lat': 'latitude'}}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions, allow_time_extrapolation = True, mesh='spherical', deferred_load=True)

#Kh_zonal = 100
#Kh_meridional = 100
#
#size2D = (fieldset.U.grid.ydim, fieldset.U.grid.xdim)
#
#fieldset.add_field(Field('Kh_zonal', data=10 * np.ones(size2D),
#                         lon=fieldset.U.grid.lon, lat=fieldset.U.grid.lat,
#                         mesh='spherical', allow_time_extrapolation=True))
#fieldset.add_field(Field('Kh_meridional', data=10 * np.ones(size2D),
#                         lon=fieldset.U.grid.lon, lat=fieldset.U.grid.lat,
#                         mesh='spherical', allow_time_extrapolation=True))

In [ ]:
class CustomParticle(ScipyParticle):  # Define a new particle class that contains three extra variables
    age = Variable('age', dtype=np.float32, initial=0.) # initialise age
    distance = Variable('distance', initial=0., dtype=np.float32)  # the distance travelled
    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
                        initial=attrgetter('lon'))  # the previous longitude
    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
                        initial=attrgetter('lat'))  # the previous latitude.
    KelpDepth = Variable('KelpDepth', dtype=np.int32) # create variable for kelp depth
    Salinity = Variable('Salinity', initial = fieldset.S) # create variable for salinity
    VelU = Variable('VelU', dtype=np.float32) # create variable for meridonal velocity
    VelV = Variable('VelV', dtype=np.float32) # create variable for zonal velocity
    KelpVelU = Variable('KelpVelU',dtype=np.float32) # create variable for meridonal kelp velocity
    KelpVelV = Variable('KelpVelV',dtype=np.float32) # create variable for zonal kelp velocity
    WindU = Variable('WindU',dtype=np.float32) # create variable for meridonal wind velocity
    WindV = Variable('WindV',dtype=np.float32) # create variable for zonal wind velocity
    bathy = Variable('bathy',dtype=np.float32) # create variable for bathymetery
    SinkSpeed = Variable('SinkSpeed',dtype=np.float32) # create variable for the sink speed of sinking kelp
    Phase = Variable('Phase', dtype=np.float32, initial = 1) # create variable to assign a number to the stage of kelp (beached, outofbounds, surface, sinking, sunk)
    Mask = Variable('Mask', initial = fieldset.mask, to_write = False) # create variable for the LandMask
    expo = Variable('expo', dtype=np.float32)
    expoW = Variable('expoW', dtype=np.float32)
    SettlingVelocity = Variable('SettlingVelocity',dtype=np.float32)
    
def SampleDistance(particle, fieldset, time): # Function measuring distance
     # Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    # Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat  # Set the stored values for next iteration. 
           
def Drag(particle, fieldset, time):
        (u,v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon] 
        particle.Salinity = fieldset.S[time, particle.depth, particle.lat, particle.lon] # partical salinity units
        so = ((particle.Salinity/1000)*0.001) # conversion of salinity to correct units
        Ac = 310.54 # cross-sectional area of plant, must be entered manually and calculated before hand in m^2
        mass = 48.65 # total mass of the kelp plant being simulated (kg)
        volume = 103829.23 # volume based on geometric shape in m^3
        kelp_density = mass/volume # kelp density in kg/m^3
        expo = kelp_density/so
        SwaterdragU = ((0.5*so*(u*u)*0.47*Ac)*expo)/mass # syntax for calculating the loss in velocity as a result of drag
        SwaterdragV = ((0.5*so*(v*v)*0.47*Ac)*expo)/mass
        waterdragU = ((0.5*so*(u*u)*0.47*Ac))/mass
        waterdragV = ((0.5*so*(v*v)*0.47*Ac))/mass
        uw = u * 0.50
        vw = v * 0.50
        expoW = 1 - expo
        winddragU = ((0.5*1.225*(uw*uw)*1.5*Ac)*expoW)/mass # syntax for calculating the loss in velocity as a result of drag
        winddragV = ((0.5*1.225*(vw*vw)*1.5*Ac)*expoW)/mass
        particle.WindU = uw
        particle.WindV = vw
        particle.VelU = u
        particle.VelV = v
        particle.expo = expo
        particle.expoW = expoW
        if particle.depth <= 1:
            particle.lon  += (u - (SwaterdragU + winddragU)) * particle.dt
            particle.lat  += (v - (SwaterdragV + winddragV)) * particle.dt
            particle.KelpVelU = (u - (SwaterdragU + winddragU))* particle.dt 
            particle.KelpVelV = (v - (SwaterdragV + winddragV))* particle.dt
            particle.Phase = 1
        if particle.depth > 1:
            particle.lon  += (u - waterdragU) * particle.dt
            particle.lat  += (v - waterdragV) * particle.dt
            particle.KelpVelU = (u - waterdragU) * particle.dt
            particle.KelpVelV = (v - waterdragV) * particle.dt
            particle.Phase = 2
                  
def Kelp(particle, fieldset, time):
        particle.bathy = fieldset.bathy[0, 0, particle.lat, particle.lon]
        V_g = (((kelp_density - so)*9.8*volume)/mass) # initial sinking velocity converted to m/s by dividing by day
        vertical_drag = (0.5*so*(V_g*V_g)*0.47*Ac)/mass # altered drag equation, no need for expo (m/s)
        BF = (volume*so*9.8)/mass
        particle.SinkSpeed = V_g - vertical_drag - BF 
        particle.SettlingVelocity = (math.sqrt(((kelp_density - so)*9.8*volume)/(0.5*so*0.47*Ac)-(volume*so*9.8)))/mass
        raft_time = 86400*31 # raft time (days) summer:24; autumn:41; winter:31; Spring: 33 
        particle.age += particle.dt # Sample age of the kelp particle
        particle.KelpDepth = particle.depth # Sample depth of the kelp particle
        if particle.age >= raft_time:
            if particle.SinkSpeed < particle.SettlingVelocity:
                particle.depth += particle.SinkSpeed 
                particle.KelpDepth = particle.depth
            else:
                particle.depth += particle.SettlingVelocity
        if particle.KelpDepth >= particle.bathy:
                particle.Phase = 3
                particle.delete()
                       
#def BrownianMotion(particle, fieldset, time):
#    """Kernel for simple Brownian particle diffusion in zonal and meridional direction.
#    Assumes that fieldset has fields Kh_zonal and Kh_meridional"""
#
#    r = 1/3.
#    kh_meridional = fieldset.Kh_meridional[time, particle.depth, particle.lat, particle.lon]
#    particle.lat += random.uniform(-1., 1.) * math.sqrt(2*math.fabs(particle.dt)*kh_meridional/r)
#    kh_zonal = fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon]
#    particle.lon += random.uniform(-1., 1.) * math.sqrt(2*math.fabs(particle.dt)*kh_zonal/r)

def Beaching(particle, fieldset, time):
    if particle.Phase == 1:
        particle.Mask = fieldset.mask[time, particle.depth,particle.lat, particle.lon]
        if particle.Mask <= 0.5:
            particle.Phase = 4
            particle.delete()
        
def DeleteParticle(particle, fieldset, time): # Function that deletes particle if it goes out of bounds to avoid OutOfBounds error
    particle.delete()
    particle.Phase = 5

In [ ]:
# Site list adapted from chapter 2 metadata, some co-ordinates have been adjusted slightly in some cases.

#site            #lon        #lat
#Soetwater       18.3375;   -34.1764
#Olifantsbos     18.3755;   -34.2564 (not included)
#Buffelsbaai     18.4702; -34.3328
#Batsata Rock    18.4802;   -34.2768
#Baboon Rock     18.4782;  -34.2552
#Miller's Point  18.4772;  -34.2303
#A-Frame         18.468;    -34.207
#Kalk Bay        18.456;    -34.121
#Kalk Bay        18.4570;  -34.1220 (updated coordinates)
#Betty's Bay     18.911;    -34.361
#Oudekraal       18.3461;   -33.9849
#Scarborough     18.3654;   -34.2387
#Hout Bay        18.3350;   -34.0622
#Kommetjie       18.3176;   -34.1501
#St Helena Bay   17.9802;  -32.7184


In [ ]:
# Define the particle set. Make sure the date corrosponds to the model being used for the simulation

# Year1 Summer: 2017, 12, 1
# Year1 Autumn: 2018, 3, 1
# Year1 Winter: 2018, 6, 1
# Year1 Spring: 2018, 9, 1
# Year2 Summer: 2018, 12, 1
# Year2 Autumn: 2019, 3, 1
# Year2 Winter: 2019, 6, 1
# Year2 Spring: 2019, 9, 1

pset = ParticleSet.from_list(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=CustomParticle,  # the type of particles (JITParticle/ScipyParticle/Custom)
                             lon=[18.3176], 
                             lat=[-34.1501],
                             repeatdt=43200,        #43200(12hrs)
                             time=(datetime(2017, 12, 1)))   

In [ ]:
## cast kernels, select appropriate 'kernel set'
#kernels = pset.Kernel(AdvectionRK4) + SampleDistance # Passive
kernels = pset.Kernel(AdvectionRK4) + SampleDistance + Drag + Kelp + Beaching #+ BrownianMotion

In [ ]:
## execute simulation

## define output file, select outfile according to simulation type

pset.execute(kernels, 
             runtime=timedelta(days=5),
             dt=timedelta(hours = 1),
             output_file=pset.ParticleFile(name="debug030.nc", outputdt=timedelta(days=1)),  
             recovery = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:
# simple plot for viewing, use for checking simulation output
#plotTrajectoriesFile('test_pas.nc')
plotTrajectoriesFile('debug030.nc')

In [ ]:
#plotTrajectoriesFile('test_pas.nc', mode = 'movie2d_notebook')
plotTrajectoriesFile('debug030.nc', mode = 'movie2d_notebook')  